# Creating a lookup table for Marching Cubes

Marching Cubes has 256 possible configurations of positive vertices within the cube, but only 22 unique configurations after symmetry is considered due to rotations and reflections.  Face descriptions for simple cases can be determined fairly quickly, but as the number of positive vertices in the configuration increases, so does the complexity.  This is where it may be more practical to consider an algorithmic approach as doing it manually leads to silly errors, not to mention tedium.

In this project, you'll develop code to generate a table of face descriptions for use with the Marching Cubes algorithm.  You will develop functions for each of the 22 unique cube configurations and generate the appropriate face descriptions accounting for symmetry and rotations.  The output will be a table of 256 entries and be suitable for use with the project , 'Visualizing Superquadrics with Marching Cubes and Marching Tetrahedra'.

Some code has been provided to help you get started.

In [ ]:
import numpy as np
import math

from test_table import *

## constants ##
DEBUG = False
X = 0
Y = 1
Z = 2

aCaseCodeMap = [None] * 256

## Part 1 - Define the cube

Before we can generate the table, we must define the configuration of the cube.  The cube is like any other cube in that it has 6 faces, 8 vertices, and 12 edges, but what we mean by defining the cube is the order in which those components are arranged as we'll need to refer to specific locations in order to define connection order for the vertices to form polygons.

The cube will be defined as having origin in the lower-front-left corner at location (0,0,0) in the XYZ coordinate space.  The vertices will be numbered in counterclockwise order around the vertical axis beginning with the bottom 4 vertices, and continuing with the top 4 vertices.  The edges will be numbered similarly, but with the vertical edges defined last, also counterclockwise around the vertical axis.  Faces will be numbered beginning with the front face and continuing counterclockwise around the cube, then follow with the bottom face and finishing with the top face.  This means Vertex 1 will be at location (1,0,0), and Vertex 6 at location (1,1,1).

![MC cube configuration](cube_config.png)

In the following cell, define arrays as tables to assist in identifying components of the cube.  These tables will be referenced by helper functions to navigate the cell quickly and consistently.  Each index of the first dimension represents a vertex of the cell.  Elements of each index should be listed in counter-clockwise order to comply with the right-hand rule (as defined when looking at the current vertex from the positive end of it's geometry normal).

- __aAdjacentMap__: Each index contains the list of 3 vertex indices connected to the current vertex by an edge.  Example: index 0 represents vertex 0 of the cube.  [4,3,1] represent the vertex indices connected by an edge to vertex 0.


- __aDiagonalMap__: Each index contains the list of 3 vertex indices located at the opposite corner of each polygon connected to the current vertex.


- __aOppositeMap__: Each index contains the vertex index located at the opposite corner of the cell.


- __aPoleTriadIndices__: Each index contains a list of 4 vertex indices which are located at the opposite corner of each polygon connected to the current vertex.  Similar to aAdjacentMap, but the first vertex index of the 4 is the current vertex.


- __aAdjacentEdgesMap__: Each index contains a list of 3 edge indices which connect the current vertex to the vertex indices listed in aAdjacentMap for the current vertex.  The edges should be listed in the same order as described in aAdjacentMap.


- __aFaceIndices__: Each index contains the list of 4 vertex indices which comprise a face (polygon).  The vertex indices for each face should be specified in the order which complies with the righthand rule.

In [ ]:
# AdjacentMap:
aAdjacentMap = [ [4,3,1], [0,2,5], [1,3,6], [7,2,0], [5,7,0], [1,6,4], [5,2,7], [6,3,4] ]

# DiagonalMap:
aDiagonalMap = []

# OppositeMap:
aOppositeMap = []

# PoleTriadIndices:
aPoleTriadIndices = []

# AdjacentEdgesMap:
aAdjacentEdgesMap = []

# FaceIndices:
aFaceIndices = []

In [ ]:
### test (not exhaustive) ###
test_adjacent_map( aAdjacentMap )
test_diagonal_map( aDiagonalMap )
test_opposite_map( aOppositeMap )
test_poletriad_map( aPoleTriadIndices )
test_adjacent_edges_map( aAdjacentEdgesMap )
test_face_indices_map( aFaceIndices )

## Part 2 - Helper functions

In the cell below, implement helper functions to assist you in generating face descriptions for a given cube configuration.  Each function is relatively short and focuses on a single discrete task.

In [ ]:
#========================================================
# isAdjacent(): determines if two vertices share a common edge.
#
# v1: first vertex index
# v2: second vertex index
# returns: True if v1 and v2 are part of the same edge, False otherwise.
#========================================================
def isAdjacent( v1, v2 ):
    return( False )

#========================================================
# isDiagonal(): determines if two vertices are on opposite corners of a face
#
# v1: first vertex index
# v2: second vertex index
# returns: True if v1 and v2 are on opposite corners of a face, False otherwise.
#========================================================
def isDiagonal( v1, v2 ):
    return( False )

#========================================================
# isOpposite(): determines if two vertices are on opposite corners of a cube
#
# v1: first vertex index
# v2: second vertex index
# returns: True if v1 and v2 are on opposite corners of the cube, False otherwise.
#========================================================
def isOpposite( v1, v2 ):
    return( False )

#========================================================
# isPolesOnSameFace(): Determines if all vertices belong to the same face of a cube
#
# aVertexIndices: indices of vertices to compare.
# returns: True if all vertices in aVertexIndices lie on the same face, False otherwise.
#========================================================
def isPolesOnSameFace( aVertexIndices ):
    return( False )

#========================================================
# isPolesOnDiagonal(): Determines if all vertices are aligned on diagonal of the cube.
#
# aVertexIndices: indices of vertices to consider
# returns: True if all vertices in aVertexIndices lie on the main diagonal of the cube, False otherwise.
#========================================================
def isPolesOnDiagonal( aVertexIndices ):

    aDiagonals = [
        [ True, False, True, False, True, False, True, False ],
        [ False, True, False, True, False, True, False, True ],
        [ True, False, False, True, False, True, True, False ],
        [ False, True, True, False, True, False, False, True ],
        [ True, True, False, False, False, False, True, True ],
        [ False, False, True, True, True, True, False, False ]
    ]
    
    nbIndices   = len( aVertexIndices )
    nbDiagonals = len( aDiagonals )
    
    for j in range( 0, nbDiagonals ):

        Count = 0

        for i in range( 0, nbIndices ):
            n = aVertexIndices[i]
            if ( aDiagonals[j][n] == True ):
                Count += 1

        if ( Count == nbIndices ):
            return( True )
        
    return( False )

#========================================================
# isPoleTriad(): Determines if the 4 specified vertices form a corner where
# one vertex is the corner, and the other 3 vertices are connected by an edge.
#
# aVertexIndices: list of vertices to consider.
# returns: True if all vertices of aVertexIndices form a corner pole, False otherwise.
#========================================================
def isPoleTriad( aVertexIndices ):
    return( False )

## Part 3 - Cube configuration functions

In the next several cells, you'll implement functions to read the bitcode for the cube configuration and return the face description of the triangle(s) which span the positive vertices for that configuration.  Face descriptions must follow the following specification:

    [N, v1, v2, ..., vN]
Where:
- __N__: Number of sides in the polygon
- __v1...vN__: vertex indices comprising the polygon

The vertices must satisfy the _right-hand rule_.  That is, when looking down the surface normal at the front face of the polygon, the vertices will be specified in the counter-clockwise direction when walking the perimeter of the polygon.  The vertices will be connected by edges, and the last vertex will automatically be connected to the first vertex to form a closed polygon.  The number of vertices specified must match N, or else the resulting polygon mesh will be corrupt.

Example: Describe a triangle comprised of vertices [0,1,2], and a quadrilateral polygon with vertices [3,4,5,6]:

    aCaseMap = [
        ...
        [3,0,1,2,4,3,4,5,6]
        ...
    ]

Triangles should share vertices and edges where possible, and the normals of the triangles should always face the positive vertices which they are most closely associated.  If adjacent triangles are coplanar (lie in the same plane forming a flat surface), merge them together to form a quadrilateral polygon.

Ah, but there's one more wrinkle - since we're creating a lookup table for use with the __Visualizing Superquadrics with Marching Cubes and Marching Tetrahedra__ project, we must make one important modification to the face descriptions.  The project expects the face descriptions to contain the edge indices of the cell where vertices were inserted, not the vertex indices.  Therefore, the above face description might be described as follows:

    aCaseMap = [
        ...
        [3,0,8,3,4,8,9,10,11]
        ...
    ]

Notice the number of sides to the polygon remains the same, but the vertex indices are replaced with the indices of the edges which contain them.  This should be read as two polygons: 
- A triangle containing vertices which reside somewhere on edges [0,8,3] (a triangle facing vertex 0)
- A quadrilateral polygon containing vertices which reside somewhere on edges [8,9,10,11] (spans all vertical edges and facing vertices [4,5,6,7]).  

If you have trouble visualizing this, pretend each vertex is inserted at the midpoint of an edge and connected in the order specified.  The surface normal will always project out from the front side of the polygon according to the right-hand rule.

__NOTE__: It is advised to begin with the simple cases (1,2,6,7 poles) before moving onto the more complex cases (3,4,5 poles).  Four poles will be the most complex as it has the most configurations to consider.

To test/debug your code, run the createTable() function at the bottom of the project, and inject the results into your Visualizing Superquadrics project for the Marching Cubes algorithm.

The __createTable()__ function at the bottom of this project parses the bitcode to determine which configuration function to call.  Each configuration function receives the bitcode and a list of vertex indices representing the positive poles as input arguments, and should return the list of edge indices which describe the polygon(s) created by the function.  Polygons should be described such that the geometry normals point toward the positive poles.  The first two configuration functions have been provided to get you started.

## One Pole

Given a cube with one positive vertex, return the face description of the generated triangle so it's normals point towards the positive vertex.

__TIP__: Don't forget about rotations and symmetry.

![one pole](mc_pole_1.png)

In [ ]:
#========================================================
# getIndicesOnePole(): get edge indices winding order for cases of one positive pole
#
# Parameters:
# code: bitCode for the cube configuration
# aVertexIndices: list of vertex indices representing the positive poles of the cube.
#
# Returns:
# list of polygon face description of the triangle(s) generated by the cube configuration.
#========================================================
def getIndicesOnePole( code, aVertexIndices ):

    aItems = []    # generated polygon description(s)
        
    aCaseCodeMap[ code ] = "1A" 

    # your code here
    
    return( aItems )

## Two Poles

Given a cube with two positive vertices, return the face description of the generated triangles so the normals point towards the positive vertices.  If the resulting triangles are coplanar, merge them into a quadrilateral polygon.

Some code has been provided to give you an idea how to approach the problem.

![two poles](mc_pole_2.png)

In [ ]:
#========================================================
# getIndicesTwoPole(): get edge indices winding order for cases of two positive poles
#
# Parameters:
# code: bitCode for the cube configuration
# aVertexIndices: list of vertex indices representing the positive poles of the cube.
#
# Returns:
# list of polygon face description(s) of the triangle(s) generated by the cube configuration.
# Each polygon is a separate entry in the list.
#========================================================
def getIndicesTwoPole( code, aVertexIndices ):

    v1 = aVertexIndices[0]
    v2 = aVertexIndices[1]

    aItems = []    # generated polygon description(s)

    # determine if vertices are adjacent
    if isAdjacent( v1, v2 ):

        # case 2A - v1 and v2 share a common edge.
        # produces a quadrilateral facing the poles
        # (12 cases)
        aCaseCodeMap[ code ] = "2A"

        # 1st vertex
        aNeighbors = aAdjacentMap[v1]

        if aNeighbors[0] == v2:
            e0 = aAdjacentEdgesMap[v1][1]
            e1 = aAdjacentEdgesMap[v1][2]
        elif aNeighbors[1] == v2:
            e0 = aAdjacentEdgesMap[v1][2]
            e1 = aAdjacentEdgesMap[v1][0]
        elif aNeighbors[2] == v2:
            e0 = aAdjacentEdgesMap[v1][0]
            e1 = aAdjacentEdgesMap[v1][1]

        # 2nd vertex
        aNeighbors = aAdjacentMap[v2]

        if aNeighbors[0] == v1:
            e2 = aAdjacentEdgesMap[v2][1]
            e3 = aAdjacentEdgesMap[v2][2]
        elif aNeighbors[1] == v1:
            e2 = aAdjacentEdgesMap[v2][2]
            e3 = aAdjacentEdgesMap[v2][0]
        elif aNeighbors[2] == v1:
            e2 = aAdjacentEdgesMap[v2][0]
            e3 = aAdjacentEdgesMap[v2][1]

        aItems.extend( [4, e0, e1, e2, e3] )
            
    elif isDiagonal( v1, v2 ):

        # Case 2B - vertices on opposite corners of same face.
        #           2 of the 3 vertex neighbors will be common to both vertices v1 and v2
        #           produces a hexagon folded down the center like a butterfly for a V-shape.
        # (12 cases)
        aCaseCodeMap[ code ] = "2B"

        # find the neighbor vertex which is not shared by the other vertex
        aNeighborsA = aAdjacentMap[v1]
        aNeighborsB = aAdjacentMap[v2]

        a = 0
        b = 0

        for i in range(0,3):
            if ( aNeighborsA[i] != aNeighborsB[0] and aNeighborsA[i] != aNeighborsB[1] and aNeighborsA[i] != aNeighborsB[2] ): 
                a = i

        for i in range(0,3):
            if ( aNeighborsB[i] != aNeighborsA[0] and aNeighborsB[i] != aNeighborsA[1] and aNeighborsB[i] != aNeighborsA[2] ): 
                b = i
                
        e0 = aAdjacentEdgesMap[v1][a]
        e1 = aAdjacentEdgesMap[v1][ (a+1) % 3 ]
        e2 = aAdjacentEdgesMap[v2][ (b+2) % 3 ]
        e3 = aAdjacentEdgesMap[v2][b]
        e4 = aAdjacentEdgesMap[v2][ (b+1) % 3 ]
        e5 = aAdjacentEdgesMap[v1][ (a+2) % 3 ]

        # start at unique neighbor on vertex i, then wind counter clockwise towards vertex v2
        aItems.extend(
            [4, e0, e1, e2, e3,     # polygon 1
            4, e3, e4, e5, e0]      # polygon 2
        )
            
    elif isOpposite( v1, v2 ):

        # case 2C - vertices are on opposite corners of the cube
        #           produces a triangle at each corner facing the corner.
        # (4 cases)
        aCaseCodeMap[ code ] = "2C"
        
        # your code here
        
    else:
        print( "no valid case: ", code )

    return( aItems )

## Three poles

Given a cube with three positive vertices, return the face description of the generated triangles so the normals point towards the positive vertices. If any triangles are coplanar, merge them into a quadrilateral polygon.

![three poles](mc_pole_3.png)

In [ ]:
#========================================================
# getIndicesThreePole(): get edge indices winding order for cases of 3 positive poles
#
# Parameters:
# code: bitCode for the cube configuration
# aVertexIndices: list of vertex indices representing the positive poles of the cube.
#
# Returns:
# list of polygon face description(s) of the triangle(s) generated by the cube configuration.
# Each polygon is a separate entry in the list.
#========================================================
def getIndicesThreePole( code, aVertexIndices ):

    aItems = []    # generated polygon description(s)

    # this may or may not be useful
    # defines which vertices of the cube are positive
    aMap = [False, False, False, False, False, False, False, False]
    aMap[ aVertexIndices[0] ] = True
    aMap[ aVertexIndices[1] ] = True
    aMap[ aVertexIndices[2] ] = True

    if isPolesOnSameFace( aVertexIndices ):

        # case 3A - all poles lie on the same face
        # produces a bent quadrilateral with one corner closer to the empty corner of the face
        aCaseCodeMap[ code ] = "3A"

        # your code here
            
    elif isPolesOnDiagonal( aVertexIndices ): 

        # case 3B - all poles lie on the cube's diagonal
        # produces a folded "U" shaped polygon (like a taco shell) with poles spanning the open side
        # v1 = middle vertex
        # v2 = corner vertex adjacent to v1
        # v3 = corner vertex opposite v2
        aCaseCodeMap[ code ] = "3B"
        
        # your code here
            
    else:
        # case 3C - all points lie in a triad plane (i.e. neighbor vertices of a corner)
        # produces a plane on each side of the triad.  
        #    - one triangle facing away from the isolated corner
        #    - a hexagon opposite the isolated corner facing the traid.
        aCaseCodeMap[ code ] = "3C"
        
        # your code here
              
    return( aItems )

## Four Poles

Given a cube with four positive vertices, return the face description of the generated triangles so the normals point towards the positive vertices. If any triangles are coplanar, merge them into a quadrilateral polygon.

Be aware of issues arising from symmetry by rotation as well as reflection.

![four poles](mc_pole_4.png)

In [ ]:
#========================================================
# getIndicesFourPole(): get edge indices winding order for cases of 4 positive poles
#
# Parameters:
# code: bitCode for the cube configuration
# aVertexIndices: list of vertex indices representing the positive poles of the cube.
#
# Returns:
# list of polygon face description(s) of the triangle(s) generated by the cube configuration.
# Each polygon is a separate entry in the list.
#========================================================
def getIndicesFourPole( code, aVertexIndices ):

    aItems = []    # generated polygon description(s)
    aMap   = [False] * 12

    if isPolesOnSameFace( aVertexIndices ):
        # case 4A - all poles lie on the same face.
        #           produces a quadrilateral from intersections among unshared edges
        # (6 cases)
        aCaseCodeMap[ code ] = "4A"
        
        # your code here

            
    elif isPoleTriad( aVertexIndices ):
        # case 4B - 1 corner plus it's 3 adjacent neighbor vertices are poles
        # (8 cases)
        aCaseCodeMap[ code ] = "4B"
        
        # your code here
        
    elif isPolesOnDiagonal( aVertexIndices ):
        # case 4C - all poles lie on the cube's diagonal
        # (6 cases: 2 top, 2 front, 2 right)
        aCaseCodeMap[ code ] = "4C"

        # your code here
        
    else:
        # cases 4D and 4E together because there is no easy way to distinguish them from the others.
        #
        # One test is to sum the neighbor poles for each of the 4 vertices.
        #      4D = 6 neighbor poles in the system
        #      4E = 4 neighbor poles in the system
        #      4F = 0 neighbor poles in the system
        
        # your code here - you'll need to figure out how to distinguish between these cases:
        aCaseCodeMap[ code ] = "4D"
        aCaseCodeMap[ code ] = "4E"
        aCaseCodeMap[ code ] = "4F"

    return( aItems )

## Five Poles

Given a cube with five positive vertices, return the face description of the generated triangles so the normals point towards the positive vertices. If any triangles are coplanar, merge them into a quadrilateral polygon.

__Hint__: compare to configurations with three positive vertices.  

![five poles](mc_pole_5.png)

In [ ]:
#========================================================
# getIndicesFivePole(): get edge indices winding order for cases of 5 positive poles
#
# Parameters:
# code: bitCode for the cube configuration
# aVertexIndices: list of vertex indices representing the positive poles of the cube.
#
# Returns:
# list of polygon face description(s) of the triangle(s) generated by the cube configuration.
# Each polygon is a separate entry in the list.
#========================================================
def getIndicesFivePole( code, aVertexIndices ):

    aItems = []    # generated polygon description(s)
    
    # your code here.  You'll need to figure out how to distinguish between the following cases:
    
    aCaseCodeMap[ code ] = "5A"
    aCaseCodeMap[ code ] = "5B"
    aCaseCodeMap[ code ] = "5C"
            
    return( aItems )

## Six Poles

Given a cube with six positive vertices, return the face description of the generated triangles so the normals point towards the positive vertices. If any triangles are coplanar, merge them into a quadrilateral polygon.

__Hint__: Compare to cube configurations of two positive vertices.

![six poles](mc_pole_6.png)

In [ ]:
#========================================================
# getIndicesSixPole(): get edge indices winding order for cases of 6 positive poles.
#
# Parameters:
# code: bitCode for the cube configuration
# aVertexIndices: list of vertex indices representing the positive poles of the cube.
#
# Returns:
# list of polygon face description(s) of the triangle(s) generated by the cube configuration.
# Each polygon is a separate entry in the list.
#========================================================
def getIndicesSixPole( code, aVertexIndices ):

    aItems = []    # generated polygon description(s)
    
    # your code here.  You must figure out how to distinguish between the following cases:
    aCaseCodeMap[ code ] = "6A"
    aCaseCodeMap[ code ] = "6B"        
    aCaseCodeMap[ code ] = "6C"

    return( aItems )

## Seven Poles

Given a cube with seven positive vertices, return the face description of the generated triangles so the normals point towards the positive vertices.

__Hint__: Compare to cube configurations of one positive vertex.

![seven poles](mc_pole_7.png)

In [ ]:
#========================================================
# getIndicesSevenPole(): get edge indices winding order for cases of seven positive poles.
#
# Parameters:
# code: bitCode for the cube configuration
# aVertexIndices: list of vertex indices representing the positive poles of the cube.
#
# Returns:
# list of polygon face description(s) of the triangle(s) generated by the cube configuration.
# Each polygon is a separate entry in the list.
#========================================================
def getIndicesSevenPole( code, aVertexIndices ):

    aItems = []    # generated polygon description(s)
    
    aCaseCodeMap[ code ] = "7A"
    
    # your code here
    
    return( aItems )

## Part 4 - Generate the table.

The function for generating the table has been provided in the cell below.  As you can see, it's fairly simple.  Iterate all 256 configurations for the cube, convert the configuration index to a bitcode, then call the appropriate function to generate the face description for the bitcode.  The face description is then recorded in the table which is printed after all configurations have been computed.  The resulting printout can be used as the Marching Cubes lookup table provided all the face descriptions are correctly defined.

__TIP__: Complete one cube configuration at a time and use the Marching Cubes algorithm to visualize the results.  Activating DEBUG at the top of the project will enable _aCaseCodeItems_, which may be useful to determine if you are using the correct logic to parse each bitcode.

In [ ]:
def createTable():
    
    aFaceDescriptions = [None] * 256

    # cubeIndices[ NbPoles ] = [ [],[],...,[] ]
    for i in range( 0, 256 ):

        aVertexIndices = []
        NbPoles = 0
        code    = 0

        # generate bitcode for the cube
        for j in range(0,8):
            if (i >> j) & 1:
                code += pow( 2, j )
                NbPoles += 1
                aVertexIndices.append(j)

        # call the appropriate face description function for the bitcode
        if   ( NbPoles == 1 ): aFaceDescriptions[i] = str( getIndicesOnePole(   code, aVertexIndices )) # 1 pole
        elif ( NbPoles == 2 ): aFaceDescriptions[i] = str( getIndicesTwoPole(   code, aVertexIndices )) # 2 poles
        elif ( NbPoles == 3 ): aFaceDescriptions[i] = str( getIndicesThreePole( code, aVertexIndices )) # 3 poles
        elif ( NbPoles == 4 ): aFaceDescriptions[i] = str( getIndicesFourPole(  code, aVertexIndices )) # 4 poles
        elif ( NbPoles == 5 ): aFaceDescriptions[i] = str( getIndicesFivePole(  code, aVertexIndices )) # 5 poles
        elif ( NbPoles == 6 ): aFaceDescriptions[i] = str( getIndicesSixPole(   code, aVertexIndices )) # 6 poles
        elif ( NbPoles == 7 ): aFaceDescriptions[i] = str( getIndicesSevenPole( code, aVertexIndices )) # 7 poles

    aItems         = []
    aCaseCodeItems = []
    
    aFaceDescriptions[0]   = [3,0,1,2]
    aFaceDescriptions[255] = [3,4,5,6]

    for i in range( 0, 256 ):
        aItems.append( str( aFaceDescriptions[i] ) + "," )
        if DEBUG == True:
            aCaseCodeItems.append( "'" + str( aCaseCodeMap[i] ) + "'" )
            
    print( "aCaseMap = [\n\t" + '\n\t'.join( aItems ) + "\n]" )            
            
    if DEBUG == True:
        aCaseCodeItems[0]   = "'0A'"
        aCaseCodeItems[255] = "'8A'"
        print( "\nvar aCaseCodeMap = [ " + ','.join( aCaseCodeItems ) + " ];" )

    return(0)

In [ ]:
aCaseCodeMap = [None] * 256

# generate the table
createTable()